### Imports

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection  import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import time
import datetime
from collections import Counter

### Reading CSV Files  

In [57]:
test_df = pd.read_csv('C:/Users/rajat/Artificial Intelligence/Project/NYTimesBlogTest.csv')
df = pd.read_csv('C:/Users/rajat/Artificial Intelligence/Project/NYTimesBlogTrain.csv')
test_df.shape

(1870, 9)

In [63]:
X.shape

(6532, 5452)

In [37]:
y = df['Popular'] 

# convert date to timestamp (milliseconds, divide by a million to get a more readable decimal)
df['PubDate'] = df['PubDate'].apply(lambda x: time.mktime(time.strptime(x, '%Y-%m-%d %H:%M:%S'))/1000000)

#replace empty values with the middle most common word

def nan_to_meaningful(x, col):
    if pd.isnull(x):
        mc = Counter(col).most_common(8)
        mc = [str(i[0]) for i in mc]
        if 'nan' in mc:
            mc.remove('nan')
        rep = mc.pop((len(mc))//2)
        return rep
        
    return x
        
my_list = ["NewsDesk","SectionName","SubsectionName","Headline","Snippet","Abstract"]

for f in my_list: 
    df[f] = df[f].apply(nan_to_meaningful, args=(df[f],))

# split sentences into series of words so each word can be a column
def string_to_series_of_words(x):    
    return pd.Series(x.split(" "))


for f in my_list[3:]:
    df[f] = df[f].apply(string_to_series_of_words)
    
    
df=pd.get_dummies(df,columns=["NewsDesk","SectionName","SubsectionName", "Headline","Snippet","Abstract"],
                  drop_first=False)

new_features = list(df.dtypes.index)
new_features.remove('Popular')

X = df[new_features]

# print(new_features)

In [38]:
X.head()

,WordCount,PubDate,UniqueID,NewsDesk_Business,NewsDesk_Culture,NewsDesk_Foreign,NewsDesk_Magazine,NewsDesk_Metro,NewsDesk_National,NewsDesk_OpEd,...,Abstract_YouTubes,Abstract_Young,Abstract_Your,Abstract_Youre,Abstract_Yunghi,"Abstract_Yusuf,",Abstract_Zachary,Abstract_Zombies,Abstract_Zun,Abstract_letter
0,508,1409.634009,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,285,1409.631247,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1211,1409.630736,3,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1405,1409.629414,4,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,181,1409.623131,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Random Forest Classifier 

In [5]:
y = df['Popular'] 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=1)

rf_model =  RandomForestClassifier(n_estimators=60)
rf_model.fit(X_train, y_train)


# Scoring based on the train RF Model
predictions = rf_model.predict(X_test)
predictions
accuracyDt = accuracy_score(y_test, predictions)

print("Accuracy on training data by Random Forest: ", accuracyDt)

Accuracy on training data by Random Forest:  0.911314984709


### Cross Validation Using Decision Tree 

In [39]:
# importing the method:
from sklearn.cross_validation import cross_val_score


decisionTree = DecisionTreeClassifier()

accuracy_list = cross_val_score(decisionTree, X, y, cv=10, scoring='accuracy')

accuracy_gb = accuracy_list.mean()

print("Accuracy Using Cross Validation With Decision Tree",accuracy_gb)

Accuracy Using Cross Validation With Decision Tree 0.763116466645


### Gradient Boosting

In [94]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
y = df['Popular'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = GradientBoostingClassifier(learning_rate= 0.01, max_depth=2,random_state=0)
clf.fit(X_train, y_train)

x1 = clf.score(X_train, y_train) 
print("Accuracy Using Cross Validation With Gradient Boosting",x1)

Accuracy Using Cross Validation With Gradient Boosting 0.882833231272


### SVM 

In [7]:
from sklearn.svm import SVC
from sklearn import svm
y = df['Popular'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)

clf.score(X_test, y_test)


0.8738518064911206

### Knn  

In [93]:
k=30
y = df['Popular'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
my_knn = KNeighborsClassifier(n_neighbors=k)
my_knn.fit(X_train, y_train)
y_predict = my_knn.predict(X_test)
accuracyKnn = accuracy_score(y_test, y_predict)
print("Accuracy using KNN", accuracyKnn)

Accuracy using KNN 0.830985915493


### Neural Networks 

In [69]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train,y_train)
y = df['Popular'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
predictions = clf.predict(X_test)
predictions
accuracyDt = accuracy_score(y_test, predictions)

print("Accuracy on training data by Neural Networks: ", accuracyDt)

Accuracy on training data by Neural Networks:  0.822448979592


### Decision Tree

In [98]:
from sklearn.tree import DecisionTreeClassifier

y = df['Popular'] 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=0)
dT = DecisionTreeClassifier()
dT.fit(X_train, Y_train)
predictions = dT.predict(X_test)
predictions
accuracyDt = accuracy_score(y_test, predictions)

print("Accuracy on training data by Decision Tree: ", accuracyDt)

Accuracy on training data by Decision Tree:  0.895897121862


### Results 